# This is a python notebook, that is able to run all models that we have made thus far. Truly One For All, no one man should have all this power

## Import liberaries

In [8]:
# import datamanagement
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
import time
import datetime
import copy

# import models
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier

# import evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score, roc_curve, classification_report, confusion_matrix
from sklearn.model_selection import RandomizedSearchCV

# import visualisation
from sklearn.tree import export_graphviz
import matplotlib.pyplot as plt
import seaborn as sns

## Import notebook

In [2]:
df_clean = pd.read_csv("../NOTEBOOKS TO REVIEW/Job/dataset_cyb&non_v2.csv", sep=';')

### Backup

In [25]:
df_copy = copy.deepcopy(df_clean)

In [70]:
df_copy.shape

(16564145, 103)

# Machine Learning Models functions

In [2]:
def now():
    now = datetime.datetime.now()   
    return now.strftime('%Y-%m-%d %H:%M:%S')

## Support Vector Machine

In [15]:
def svm_model(train,test,train_labels,test_labels):
    #train model
    clf = svm.SVC(kernel='linear')
    clf.fit(train, train_labels)
    
    #test model
    y_pred = clf.predict(test)
    score = classification_report(y_pred, test_labels, output_dict=True)
    df_score = pd.DataFrame(score).transpose()
    df_score.to_csv('svm_score_final_run_'+now()+'.csv', sep=";", index=False)
    
    #saving TP
    SVMTest = test.copy()
    SVMTest['Predicted'] = y_pred
    SVMTest['Actual'] = test_labels
    SVMTest = SVMTest.loc[SVMTest['Predicted'] == SVMTest['Actual']]
    SVM_Predictions = data.iloc[list(SVMTest.index.values.tolist())]
    SVM_Predictions.to_csv('svm_TP_final_run_'+now()+'.csv', sep=";", index=False)
      
    #postitive target feature importance
    svm_feature_result = pd.DataFrame({'feature': train.columns,'importance': clf.coef_[0]}).sort_values('importance', ascending = False)
    svm_feature_result.to_csv('svm_featureimp_final_run_'+now()+'.csv', sep=";", index=False)

## Random Forest

In [27]:
def rf_model(train,test,train_labels,test_labels):
    #train model
    model = RandomForestClassifier(n_estimators=29, max_features = 0.7999999999999999, n_jobs=-1, verbose = 1, max_depth = 10, bootstrap = True)
    model.fit(train, train_labels)
   
    #test model
    rf_predictions = model.predict(test)
    score = classification_report(rf_predictions, test_labels, output_dict=True)
    df_score = pd.DataFrame(score).transpose()
    df_score.to_csv('rf_score_final_run_'+now()+'.csv', sep=";", index=False)
    
    #saving TP
    RFTest = test.copy()
    RFTest['Predicted'] = rf_predictions
    RFTest['Actual'] = test_labels
    RFTest = RFTest.loc[RFTest['Predicted'] == RFTest['Actual']]
    RF_Predictions = data.iloc[list(RFTest.index.values.tolist())]
    RF_Predictions.to_csv('rf_TP_final_run_'+now()+'.csv', sep=";", index=False)

    #postitive target feature importance
    rf_feature_result = pd.DataFrame({'feature': train.columns,'importance': model.feature_importances_}).sort_values('importance', ascending = False)
    rf_feature_result.to_csv('rf_featureimp_final_run_'+now()+'.csv', sep=";", index=False)

## Decision tree with Gradient Boosting

In [21]:
def dtgb_model(train,test,train_labels,test_labels):
    #train model
    model = GradientBoostingClassifier(n_estimators=100, max_depth=4, subsample=0.99, learning_rate=0.977)
    model.fit(train, train_labels)
    
    #test model
    dtgb_predictions = model.predict(test)
    score = classification_report(dtgb_predictions, test_labels, output_dict=True)
    df_score = pd.DataFrame(score).transpose()
    df_score.to_csv('/Job/run/dtgb_score_final_run_'+now()+'.csv', sep=";", index=False)
    
    #saving TP
    DTGBTest = test.copy()
    DTGBTest['Predicted'] = dtgb_predictions
    DTGBTest['Actual'] = test_labels
    DTGBTest = DTGBTest.loc[DTGBTest['Predicted'] == DTGBTest['Actual']]
    DTGB_Predictions = data.iloc[list(DTGBTest.index.values.tolist())]
    DTGB_Predictions.to_csv('/run/dtgb_final_TP_final_run_'+now()+'.csv', sep=";", index=False)

    #postitive target feature importance
    dtgb_feature_result = pd.DataFrame({'feature': train.columns,'importance': model.feature_importances_}).sort_values('importance', ascending = False)
    dtgb_feature_result.to_csv('/run/dtgb_featureimp_final_run_'+now()+'.csv', sep=";", index=False)

## Logistic Regression

In [22]:
def logreg_model(train,test,train_labels,test_labels):
    #train model
    logmodel = LogisticRegression(solver = 'lbfgs')
    logmodel.fit(train, train_labels)
    
    #test model
    predictions = logmodel.predict(test)
    score = classification_report(predictions, test_labels, output_dict=True)
    df_score = pd.DataFrame(score).transpose()    
    df_score.to_csv('/run/logreg_score_final_run_'+now()+'.csv', sep=";", index=False)
    
    #saving TP
    LRTest = test.copy()
    LRTest['Predicted'] = predictions
    LRTest['Actual'] = test_labels
    LRTest = LRTest.loc[LRTest['Predicted'] == LRTest['Actual']]
    LR_Predictions = data.iloc[list(LRTest.index.values.tolist())]
    LR_Predictions.to_csv('/run/logreg_final_TP_final_run_'+now()+'.csv', sep=";", index=False)
    
    #postive target feature importance
    logreg_feature_result = pd.DataFrame({'feature': train.columns,'importance': logmodel.coef_[0]}).sort_values('importance', ascending = False)
    logreg_feature_result.to_csv('/run/logreg_featureimp_final_run_'+now()+'.csv', sep=";", index=False)

### Variable declaration

In [31]:
df_non = df_copy.loc[df_copy['is_cyber_victim_1'] == 0]
df_cyber = df_copy.loc[df_copy['is_cyber_victim_1'] == 1]
sample_size = len(df_cyber)
loop_size = 1

### Running model mulitple times

In [28]:
for i in range(1):
    #sampling data
    data = pd.read_csv("stratified_TP_data_2020-01-14 19:06:54.csv", sep=';')
    
    #splitting data
    labels = np.array(data.pop('is_cyber_victim_1'))
    train, test, train_labels, test_labels = train_test_split(data, labels, stratify = labels, test_size = 0.3, random_state = 21)
    
    #running models
    #dtgb_model(train, test, train_labels, test_labels)
    #logreg_model(train, test, train_labels, test_labels)
    rf_model(train, test, train_labels, test_labels)   
    #svm_model(train, test, train_labels, test_labels)
    
    #merging TP
    #frames = [SVM_Predictions, RF_Predictions, LR_Predictions]
    #CombinedPredictions = pd.concat(frames, sort=False)
    #CombinedPredictions.sort_index(inplace=True)
    #CombinedPredictions.to_csv('combined_TP_run_'+now()+'.csv', sep=";", index=False)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  29 out of  29 | elapsed:    4.2s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  29 out of  29 | elapsed:    0.4s finished


In [49]:
svm.columns.values

array(['migr_background_Eerste generatie migratieachtergrond',
       'migr_background_Geen migratieachtergond',
       'migr_background_Tweede generatie migratieachtergrond',
       'sex_Mannen', 'sex_Vrouwen', 'unemploy_benefit_ ',
       'unemploy_benefit_Heeft in betreffend jaar minimaal een maand een WW-uitkering ontvangen',
       'debts_ ', 'debts_Heeft WSNP traject in betreffend jaar',
       'laborer_cath_ ',
       'laborer_cath_Behoort tot huishouden zonder waargenomen inkomen',
       'laborer_cath_Directeur-grootaandeelhouder',
       'laborer_cath_Meewerkend gezinslid',
       'laborer_cath_Nog niet schoolgaand/scholier/student met inkomen',
       'laborer_cath_Nog niet schoolgaand/scholier/student zonder inkomen',
       'laborer_cath_Ontvanger bijstandsuitkering',
       'laborer_cath_Ontvanger pensioenuitkering',
       'laborer_cath_Ontvanger uitkering sociale voorziening overig',
       'laborer_cath_Ontvanger uitkering ziekte/arbeidsongeschiktheid',
       'laborer

In [12]:
svm = pd.read_csv("../David/final svm/svm_TP_run_2020-01-14 12:51:27.csv", sep=';')
dtgb = pd.read_csv("dtgb_TP_run_2020-01-14 16:33:11.csv", sep=';')
rf = pd.read_csv("rf_TP_run_2020-01-14 16:33:20.csv", sep=';')
lr = pd.read_csv("logreg_TP_run_2020-01-14 16:33:15.csv", sep=';')

frames = [svm, dtgb, rf, lr]
allmodels = pd.concat(frames, sort=False)
allmodels.sort_index(inplace=True)
allmodels.to_csv('combined_TP_run_'+now()+'.csv', sep=";", index=False)

In [54]:
df_no = pd.read_csv("first_sample_2020-01-09 11:58:10.csv", sep=';')
df_sample_non = df_no.loc[df_no['is_cyber_victim_1'] == 0]

In [64]:
allmodels['is_cyber_victim_1'] = 1

In [67]:
len(allmodels)

102096

In [69]:
len(df_sample_non)

88131

In [66]:
for i in range(loop_size):
    #sampling data
    frame2 = [allmodels, df_sample_non.sample(len(CombinedPredictions))]
    data2 = pd.concat(frame2, sort=False, ignore_index=True)
    data2.to_csv('TP_sample'+now()+'.csv', sep=";", index=False)
    
    #splitting data
    labels = np.array(data2.pop('is_cyber_victim_1'))
    train, test, train_labels, test_labels = train_test_split(data2, labels, stratify = labels, test_size = 0.3, random_state = 21)
    
    #running models
    logreg_model(train, test, train_labels, test_labels)
    rf_model(train, test, train_labels, test_labels)
    svm_model(train, test, train_labels, test_labels)
    
    #mering TP
    #frames = [SVM_Predictions, RF_Predictions, LR_Predictions]
    #CombinedPredictions = pd.concat(frames, sort=False)
    #CombinedPredictions.sort_index(inplace=True)
    #CombinedPredictions.to_csv('combined_TP_run_two.csv', sep=";", index=False)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').